# ReasoningAgnet (Tree of Thoughts with Beam Search)

## Introduction

This repository demonstrates how to implement a Tree of Thoughts (ToT) reasoning agent using beam search in under 50 lines of core code (excluding auxiliary components like visualization and logging). The `ReasoningAgent` leverages large language models (LLMs) such as GPT-4 or Llama to perform complex reasoning tasks by exploring multiple thought trajectories.

In the provided example, we showcase how the `ReasoningAgent`, even when using a smaller model (e.g., `gpt-4o-mini`), can effectively generate and evaluate multiple reasoning paths to solve intricate problems.

## What is the Tree of Thoughts and Beam Search?

The **Tree of Thoughts (ToT)** is a reasoning framework where an agent considers multiple possible reasoning steps (thoughts) and organizes them into a tree structure. **Beam search** is an optimization algorithm that explores a subset of the tree by keeping the top `k` options at each level (the beam size). Combining ToT with beam search allows the agent to efficiently navigate through the most promising thought trajectories.

Use **Beam Size = 1** for a special case of O1-style reasoning, which is similar to CoT with multiple prompts. 


## Key Features

- **Enhanced Problem-Solving**: Implements the Tree of Thoughts framework to improve reasoning capabilities.
- **Beam Search Optimization**: Utilizes beam search to efficiently explore and evaluate multiple thought paths.
- **Multi-Agent Collaboration**: Includes a thinker agent to generate possible next steps and a grader agent to evaluate them.
- **Visualization Support**: Provides visualization of the thought tree using Graphviz for better analysis.
- **Customizable Parameters**: Allows configuration of maximum depth, beam size, and answer selection approach.

In [ ]:
config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": "API_KEY",
    }
]
verbose = False

In [13]:
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.reasoning_agent import ReasoningAgent

thought_agent = ReasoningAgent(
    name="thought_agent",
    llm_config={"config_list": config_list},
    verbose=verbose,
    beam_size=1,  # using beam size 1 for special case of O1-style reasoning.
)
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
    max_consecutive_auto_reply=10,
)

In [14]:
question = "Design a supply chain system for a coffee roasting company."


def last_meaningful_msg(sender, recipient, summary_args):
    import warnings

    if sender == recipient:
        return "TERMINATE"

    summary = ""
    chat_messages = recipient.chat_messages[sender]

    for msg in reversed(chat_messages):
        try:
            content = msg["content"]
            if isinstance(content, str):
                summary = content.replace("TERMINATE", "")
            elif isinstance(content, list):
                # Remove the `TERMINATE` word in the content list.
                summary = "\n".join(
                    x["text"].replace("TERMINATE", "") for x in content if isinstance(x, dict) and "text" in x
                )
            if summary.strip().rstrip():
                return summary
        except (IndexError, AttributeError) as e:
            warnings.warn(f"Cannot extract summary using last_msg: {e}. Using an empty str as summary.", UserWarning)
    return summary

In [10]:
ans = user_proxy.initiate_chat(thought_agent, message=question, summary_method=last_meaningful_msg)

user_proxy (to thought_agent):

Design a supply chain system for a coffee roasting company.

--------------------------------------------------------------------------------
len(final_answers) 0
len(prev_leafs) 1
thought_agent (to tot_thinker):

# Question: Design a supply chain system for a coffee roasting company.
---
What are the possible next steps?

--------------------------------------------------------------------------------
tot_thinker (to thought_agent):

**Reflection**
The previous steps involved an initial inquiry into designing a supply chain for a coffee roasting company, but no concrete actions or progress have been documented. It's important to clearly define the components and challenges of the supply chain, such as sourcing beans, roasting processes, packaging, and distribution. Without these details, it's challenging to create an effective plan. 

**Possible Options:**
Option 1: Conduct a needs assessment to identify the specific requirements and constraints of the 

In [11]:
print(ans.summary)

To design a supply chain system for a coffee roasting company, we can follow a systematic approach as outlined in the thought process. Below is a detailed breakdown of the steps involved in creating this supply chain system:

### Step 1: Conduct a Needs Assessment
1. **Identify Suppliers**: Determine who the suppliers of raw coffee beans will be (e.g., local farms, international imports).
2. **Logistics Requirements**: Assess transportation methods for both incoming beans and outgoing roasted coffee.
3. **Roasting Facilities**: Evaluate the infrastructure needed for roasting, such as roasting machines, warehouses for green beans, and finished product storage.
4. **Distribution Channels**: Identify how the roasted coffee will be delivered to customers (e.g., direct sales, retailers, online).
5. **Quality Control**: Establish the necessary quality assurance processes throughout the supply chain.

### Step 2: Analyze the Needs Assessment Results
- **Prioritize Key Areas**: Based on the as

### ReasoningAgent with Nested Chats

In [ ]:
writer = AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional writer, known for your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    You should improve the quality of the content based on the feedback from the user.
    """,
)

def reflection_message(recipient, messages, sender, config):
    print("Reflecting...", "yellow")
    return f"Reflect, Reason and provide critique on the following writing. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


In [16]:

user_proxy.register_nested_chats(
    [{"recipient": thought_agent, "message": reflection_message, "summary_method": "last_msg", "max_turns": 1}],
    trigger=writer,
)

In [17]:
task = """Write a concise but engaging blogpost about Navida."""

res = user_proxy.initiate_chat(recipient=writer, message=task, max_turns=2, summary_method="last_msg")

user_proxy (to Writer):

Write a concise but engaging blogpost about Navida.

--------------------------------------------------------------------------------
Writer (to user_proxy):

### Discovering Navida: The Future of Travel Experiences

In an age where travel has become both a routine necessity and a cherished escapade, a new player has emerged on the scene—Navida. This innovative platform is redefining how we experience travel, transforming the mundane into the extraordinary.

#### What is Navida?

Navida is more than just a travel app; it’s a holistic travel experience that tailors itineraries based on individual preferences, interests, and even moods. By leveraging cutting-edge technology and an intuitive user experience, Navida crafts personalized journeys that resonate with the traveler, whether they seek adventure, relaxation, or cultural immersion.

#### Why Choose Navida?

1. **Personalized Recommendations**: Unlike traditional platforms that offer generic packages, Navida

KeyboardInterrupt: 